<a href="https://colab.research.google.com/github/JROBOTO/Abalone/blob/master/Learning_WORKING_AIRSTRIKER_new_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install the required dependencies

In [0]:
!apt-get install pkg-config lua5.1 build-essential libav-tools git

# rendering guide from https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/
# https://mikelyons.org/2018/05/25/Running-Retro-on-Google-Colab.html
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  ffmpeg

E: Package 'libav-tools' has no installation candidate
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (40.8.0)


In [0]:
# for retro install
!pip install tqdm retrowrapper gym-retro
!pip install -U git+git://github.com/frenchie4111/dumbrain.git

    100% |████████████████████████████████| 162.0MB 213kB/s 
  Cloning git://github.com/frenchie4111/dumbrain.git to /tmp/pip-req-build-qr1spooi
  Stored in directory: /tmp/pip-ephem-wheel-cache-pg13or07/wheels/50/8e/6f/47c68c95113aa8c02ac02bde75673ace7c3d3636842c75fcb6
Successfully built dumbrain


# Step 2: Install your games

In [0]:
# Don't run this command unless you already own the games, otherwise you are pirating :)
#!python -m dumbrain.rl.retro_contest.install_games http://aiml.mikelyons.org/datasets/sonic/Sonic%20Roms.zip 

# Step 3: Create an env using retrowrapper


In [0]:
import retro
import retrowrapper
# for rendering 

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
#eg screen resolution 1400x900
import matplotlib.pyplot as plt
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

#env = retro.make(game='SonicTheHedgehog-Genesis', state='GreenHillZone.Act2')



# Step 4: Play the game

In [0]:
import gym
import gym.spaces
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
from collections import deque
import numpy as np
from skimage import transform
from collections import deque
import retro
from skimage.color import rgb2grey
from skimage.color import rgb2hsv
import random
import cv2


'''
Gym provides the ability to wrap controllers and override parent classes to control how actions 
and rewards are handled.  
'''


class ActionController(gym.ActionWrapper):

    def __init__(self, environment):
        super(ActionController, self).__init__(environment)

        buttons = ["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
        actions = [['LEFT'], ['RIGHT'], ['LEFT', 'B'], ['RIGHT', 'B'], ['B']]
        # TODO test combo actions - ['LEFT', 'DOWN'] ['LEFT', 'DOWN'] ['LEFT', 'DOWN'] - ?

        self._actions = []
        for action in actions:
            # Turn off all actions
            arr = np.array([False] * 12)

            for button in action:
                arr[buttons.index(button)] = True
            self._actions.append(arr)

        # sets the number of spaces to pick an action from (7)
        self.action_space = gym.spaces.Discrete(len(self._actions))

    def action(self, a):
        return self._actions[a].copy()


class RewardScaler(gym.RewardWrapper):
    """
    Scales rewards from environment to be more consumable.
    TODO More adjustments can be made here
    """

    def reward(self, reward):
        return reward * 0.01
   
    
#

"""
This is a basic memory management systems which uses deque to store experiences. When the max memory size it met, old
memories are discarded and the new memory is added to the front.

based off: https://github.com/udacity/deep-learning/blob/master/reinforcement/Q-learning-cart.ipynb
"""

class Memory():
    def __init__(self, max_memory_size):
        self.buffer = deque(maxlen=max_memory_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        index = np.random.choice(np.arange(len(self.buffer)),
                                 size=batch_size,
                                 replace=False)

        return [self.buffer[ii] for ii in index]

    # to prevent from sampling empty memory we need to know the size
    def get_size(self):
        return len(self.buffer)

 


class DQN():
   
    def conv_net(self, inputs):
        conv1 = keras.layers.Conv2D(
            filters=32,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)(inputs)
        
        
        pool1 = keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)(conv1)

        conv2 = keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)(pool1)
        
        pool2 = keras.layers.MaxPool2D( pool_size=[2, 2], strides=2)(conv2)
        
        conv3 = keras.layers.Conv2D(
            filters=128,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)(pool2)
        
        pool3 = keras.layers.MaxPool2D( pool_size=[2, 2], strides=2)(conv3)
        
        #print(pool3.shape)
        pool2_flat = tf.reshape(pool3, [-1, 13 * 10 * 128])
        dense = keras.layers.Dense(units=1024, activation=tf.nn.relu)(pool2_flat,)
        dropout = keras.layers.Dropout(rate=1-0.4)(dense)
        # training=tf.estimator.ModeKeys.TRAIN

        # Logits Layer
        logits = keras.layers.Dense(units=5)(dropout)
        
        return logits

    def __init__(self, state_size, action_size, learning_rate, name='DQN', ENABLE_FAST_VERSION=False):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate

        with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
            #shape of input = [batch, in_height, in_width, in_channels]
            #shape of filter = [filter_height, filter_width, in_channels, out_channels]
            # Creating placeholders
            self.inputs = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
            self.actions = tf.placeholder(tf.float32, [None, self.action_size], name="actions")  # TODO replace # with action space
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")

            self.global_step = tf.Variable(0, name='global_step', trainable=True)
            self.output = self.conv_net(self.inputs)

            # Q is our predicted Q value.
            self.Q = tf.reduce_sum(tf.multiply(self.actions, self.output))

            # The loss is the difference between our predicted Q_values and the Q_target
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q), name="loss")
            tf.summary.scalar("loss", self.loss)
            self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)


In [0]:

TRAINING_ACTIVE = True
RENDER_ACTIVE = False

EPISODES = 20 # 5
MAXTIMESTEPS = 40000  # TODO is this enough? this is close to 10 min
BATCH_SIZE = 64  # TODO optimise? 64

LEARNING_RATE = 0.0001

DISCOUNT_FACTOR = 0.4 #0.99  # TODO 0.99? 0.4


EPSILON_START = 0.3  # 0.4
EPSILON_END = 0.01
EPSILON_DECAY = 0.0001

IMAGE_WIDTH = 110  # 120
IMAGE_HEIGHT = 84  # 84
FRAMES_STACKED = 8

ACTION_SIZE = 5  # THIS GETS SET AGAIN AFTER ENVIRONMENT IS SET UP
STATE_SIZE = [IMAGE_WIDTH, IMAGE_HEIGHT, FRAMES_STACKED]

MEMORY_SIZE = 15000

# Initialize deque with zero-images one array for each image
stacked_frames = deque([np.zeros((IMAGE_WIDTH, IMAGE_HEIGHT), dtype=np.int) for i in range(FRAMES_STACKED)],
                       maxlen=FRAMES_STACKED)


# Stats
old_score = 0
old_lives = 3

lengths = []
pltRewards = []

def main():
    # GPU TESTING
    # if tf.test.is_built_with_cuda():
    #     print("The installed version of TensorFlow includes GPU support.")
    # else:
    #     print("The installed version of TensorFlow does not include GPU support.")

    run_model()


def preprocess_frame(frame, restart):
    """
    This method reduces the problems space. First grey scaling the frame, normalising it and shrinking it
    Then stacking frames to create the concept of motion.
    """
    grey_image = rgb2grey(frame)
    norm_image = grey_image / 255
    resized_image = transform.resize(norm_image, [IMAGE_WIDTH, IMAGE_HEIGHT], order=1)

    # Need to call the global definition
    global stacked_frames

    # stack frames - give sense of motion
    if restart:
        # clear the stack (zero out each frame for each frame saved)
        stacked_frames = deque([np.zeros((IMAGE_WIDTH, IMAGE_HEIGHT), dtype=np.int) for i in range(FRAMES_STACKED)],
                               maxlen=FRAMES_STACKED)

        # Make a stack of the same position
        for i in range(FRAMES_STACKED):
            stacked_frames.append(resized_image)

        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)

    else:
        stacked_frames.append(resized_image)
        stacked_state = np.stack(stacked_frames, axis=2)

    return stacked_state, stacked_frames


def get_action(state, actions, epsilon, session, network):
    """
        This method returns the action to take in the next state
    """

    # TODO improve decay rate
    # We need the exploration to go up when stagnation occurs.
    epsilon = np.maximum(epsilon - EPSILON_DECAY, EPSILON_END)

    if random.random() > 1 - epsilon:
        # action = np.random.choice(actions)
        #print("random action")
        choice = random.randint(0, len(actions) - 1)  # possible action space.
    else:
        #print("Best action")
        # Estimate the new Q values based on current state
        Qs = session.run(network.output, feed_dict={
            network.inputs: state.reshape((1, *state.shape))})  # TODO are these the right inputs for this? # todo 1
        # exploit: take the best action from the DQL
        choice = np.argmax(Qs)
        # action = possible_actions[int(choice)] TODO using choice instead of action

    return choice, epsilon


def get_environment():
    """
        This method sets up the environment to be used. This includes creating a custom
        action controller as well as loading the game rom and altering reward systems.

        Retro provides a baseline to be used to start adjusting the environment. this includes
        wrapper tools made to allow backtracking in the environment.
    """

    # Create environment
    # Level notes: It gets stuck at a point where it needs to go back stand on a rock and jump off the rock
    # environment = retro.make(game='SonicTheHedgehog-Genesis', state='GreenHillZone.Act1')

    # Level notes: It needs to jump on an enemy underwater to get higher
    # environment = retro.make(game='SonicTheHedgehog-Genesis', state='LabyrinthZone.Act1')

    # Level notes: It needs a spin to get up a spin path
    # environment = retro.make(game='SonicTheHedgehog-Genesis', state='GreenHillZone.Act2')

    # Level notes: It needs a spin to get up a spin path
    environment = retro.make(game='Airstriker-Genesis')
    
    environment=wrap_env(environment)
    
    # set up action controller
    environment = ActionController(environment)
    possible_actions = np.array(np.identity(environment.action_space.n, dtype=np.int).tolist())

    return environment, possible_actions


def modify_reward(reward, info, steps):
    global old_score
    global old_lives

    score = info['score']
    gameover = info['gameover']
    lives = info['lives']

    # by using score we can encourage it to kill enemies - score and rings are different
    if score > old_score:
        old_score = score
        reward += 20.0
    
    
    if lives < old_lives:
        old_lives = lives
        reward -= 40.0

    return reward*0.1



def get_status(rewards_collection, episode, step, xposition):
    # early episodes it should not worry about stuck status
    if not (episode > 5):
        return False

    # TODO check if this is actually a decent guess
    if np.average(rewards_collection[:100]) < 1:
        # Need to keep track of if it is always getting stuck in the same place
        stuck_position.append(episode, xposition)

def reset_globals():
    global old_score
    global old_lives
    
    old_score = 0
    old_lives = 3

    
def run_model():
    # the 7 is the new amount of actions space but the environment isn't fully set up here.
    network = DQN(STATE_SIZE, ACTION_SIZE, LEARNING_RATE)

    # To save the model
    saver = tf.train.Saver()

    if TRAINING_ACTIVE:
        with tf.Session() as sess:
            # try keep restoring
            # saver.restore(sess, "./model.ckpt")

            memory = Memory(MEMORY_SIZE)

            # Initialize the variables
            sess.run(tf.global_variables_initializer())

            # Set up environment to allow exploring backwards and an action controller
            environment, possible_actions = get_environment()
            
            epsilon = EPSILON_START
          
            for episode in range(EPISODES):
                print("EPISODE ", episode)
                # Init
                done = False
                epLength = 0
                step = 0
                
                reset_globals()

                # Initialize the rewards of the episode
                episode_rewards = []
                # total_reward = np.sum(episode_rewards)

                # first state
                frames = environment.reset()
                #print("observation: {}".format(frames.shape))
                state, stacked_frames = preprocess_frame(frames, True)

                while not done:
                    if RENDER_ACTIVE:
                        environment.render()

                    # TODO NOTE: A step should be the time to preform one action NOT a frame.
                    # TODO stop jumping so much.
                    #  If the actions are up right left right down right - all of this will be in the air
                    # Predict the action to take and take it
                    if (step%20==0):
                        action, epsilon = get_action(state, possible_actions, epsilon, sess, network)
                    
                    
                    # Take the action
                    frames, reward, done, info = environment.step(action)

                    # provide custom rewards based on the info data file - eg rewards for killing enemies
                    #reward = modify_reward(reward, info, step)

                    # Add the reward to total reward
                    episode_rewards.append(reward)

#                   stuck = get_status(episode_rewards, episode, step, info['x'])

#                   if stuck:
#                       print("stuck")
#                       step + 100

                    # this is for debugging
                    if (step % 100 == 0):
                        #print(epsilon)
                        #print(action)
                        #print(episode_rewards[-9:])
                        epLength += 1
                        total_reward = np.sum(episode_rewards)
                        print("Total rewards: ", total_reward)
                        #print(step)
                        

                    # roughly 5 min of game play
                    if step >= MAXTIMESTEPS:
                        done = True
                        

                    if done:
                        # If it dies we need to reset the step count
                        # step = 0
                        # the episode ends so go next state
                        next_state = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.int)
                        next_state, stacked_frames = preprocess_frame(next_state, False)

                        # Get the total reward of the episode
                        total_reward = np.sum(episode_rewards)
                        print("Total rewards: ", total_reward)
                        print("Episode length ", epLength)
                        lengths.append(epLength)
                        pltRewards.append(total_reward)

                        memory.add((state, action, reward, next_state, done))

                    else:
                        # Get the next state
                        next_state, stacked_frames = preprocess_frame(frames, False)

                        # Add experience to memory
                        memory.add((state, action, reward, next_state, done))

                        # st+1 is now our current state
                        state = next_state

                    # When we have added enough memories we can start to learn from them
                    if memory.get_size() > BATCH_SIZE and (step % 100 == 0):  # TODO how many steps?
                        #print("MEMSAMPLE")
                        # Obtain random mini-batch from memory
                        batch = np.array(memory.sample(BATCH_SIZE))  # TODO min *(memory.get_size(), BATCH_SIZE)
                        states_mb = np.array([each[0] for each in batch], ndmin=3) 
                        # TODO ValueError: Cannot feed value of shape (64,) for Tensor 'DQN/actions_:0',
                        #  which has shape '(?, 3)'
                        # actions_mb = np.array([each[1] for each in batch])
                        actions_mb = np.zeros((BATCH_SIZE, environment.action_space.n))
                        for i in range(BATCH_SIZE):
                            actions_mb[action] = action
                            # print("action_mb:", actions_mb)

                        rewards_mb = np.array([each[2] for each in batch])

                        next_states_mb = np.array([each[3] for each in batch], ndmin=3)

                        dones_mb = np.array([each[4] for each in batch])

                        target_Qs_batch = []

                        # Get Q values for next_state
                        Qs_next_state = sess.run(network.output, feed_dict={network.inputs: next_states_mb})

                        # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + gamma*maxQ(s', a')
                        for i in range(0, len(batch)):
                            terminal = dones_mb[i]

                            # If we are in a terminal state, only equals reward
                            if terminal:
                                target_Qs_batch.append(rewards_mb[i])

                            else:
                                target = rewards_mb[i] + DISCOUNT_FACTOR * np.max(Qs_next_state[i])
                                target_Qs_batch.append(target)

                        targets_mb = np.array([each for each in target_Qs_batch])

                        loss, _ = sess.run([network.loss, network.optimizer],
                                           feed_dict={network.inputs: states_mb,
                                                      network.target_Q: targets_mb,
                                                      network.actions: actions_mb})

                    step += 1

                if episode % 100 == 0:
                    saver.save(sess, "./model.ckpt")
                    print("Model Saved")

    else:
        # Load the model
        # play the game
        with tf.Session() as sess:
            saver.restore(sess, "./model.ckpt")
            # TODO make a playing state


if __name__ == '__main__':
    main()

#if KeyboardInterrupt: 
#    done = True    

plt.figure(figsize=(10, 5))
plt.plot(range(EPISODES), lengths, linewidth=2.0)
plt.xlabel("epsiodes")
plt.ylabel("length of episode")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range(EPISODES), pltRewards, linewidth=2.0)
plt.xlabel("epsiodes")
plt.ylabel("epsiode rewards")
plt.legend()
plt.grid()
plt.show()

show_video()

EPISODE  0


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Total rewards:  0.0
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  40.0
Total rewards:  40.0
Episode length  16
Model Saved
EPISODE  1


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Total rewards:  0.0
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  40.0
Total rewards:  40.0
Episode length  14
EPISODE  2
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  20.0
Total rewards:  40.0
Total rewards:  40.0
Total rewards:  40.0
Total rewards:  40.0
Total rewards:  60.0
Total rewards:  60.0
Total rewards:  60.0
Total rewards:  60.0
Total rewards:  80.0
Total rewards:  80.0
Total rewards:  80.0
Total rewards:  80.0
Total rewards:  80.0
Total rewards:  80.0
Total rewards:  100.0
Total rewards:  100.0
Episode length  23
EPISODE  3
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  0.0
Total rewards:  20.0
Total rewards:  20.0
Tota